Rename a bunch of files using Git and renamer?



In [3]:
var execSync = require('child_process').execSync;
try {
    require.resolve('renamer');
} catch (e) {
    execSync('npm install renamer');
}
var renamer = require('renamer');
var renameUsingGit = (root, match, find, replace) => {
    var files = renamer.expand(path.join(root, match));
    var results = renamer.replace({
        files: files.filesAndDirs, 
        find: find, 
        replace: replace});
    return renamer.dryRun(results).list.map(r => {
        // rename with git instead
        var output = execSync('git ' 
                 + '--work-tree=' + JSON.stringify(root)
                 + ' --git-dir=' + JSON.stringify(path.join(root, '.git'))
                 + ' mv ' + JSON.stringify(r.before) 
                 + ' ' + JSON.stringify(r.after));
        return output.toString() || r.after;
    });
};
(renameUsingGit);


[Function: renameUsingGit]

In [4]:
// test it
// rename all scss files to less
renameUsingGit(
    '/Users/briancullinan/Documents/portal',
    'src/**/*.scss',
    '.scss',
    '.less');

[ '/Users/briancullinan/Documents/portal/src/app/app.component.less',
  '/Users/briancullinan/Documents/portal/src/app/app.less',
  '/Users/briancullinan/Documents/portal/src/app/auth/login/login.component.less',
  '/Users/briancullinan/Documents/portal/src/app/auth/register-form/register-form.component.less',
  '/Users/briancullinan/Documents/portal/src/app/layout/footer/footer.component.less',
  '/Users/briancullinan/Documents/portal/src/app/layout/layout.component.less',
  '/Users/briancullinan/Documents/portal/src/app/layout/menu/menu.component.less',
  '/Users/briancullinan/Documents/portal/src/app/layout/not-found/not-found.component.less',
  '/Users/briancullinan/Documents/portal/src/app/layout/toolbar/toolbar.component.less',
  '/Users/briancullinan/Documents/portal/src/app/temp-demo-pages/contact/contact.component.less',
  '/Users/briancullinan/Documents/portal/src/app/temp-demo-pages/demo/demo.component.less',
  '/Users/briancullinan/Documents/portal/src/app/temp-demo-pages/h

How to use memory-fs and rewire to audit cli events?



In [2]:
var path = require('path');
var execSync = require('child_process').execSync;
try {
    require.resolve('@angular/cli');
    require.resolve('rewire');
    require.resolve('typescript')
    require.resolve('memory-fs');
} catch (e) {
    execSync('npm install @angular/cli rewire typescript memory-fs');
}
var rewire = require('rewire');

var mockAllWithFallback = (fallback, map) => {
    var f = Object.assign({}, require(fallback));
    Object.keys(f).forEach(k => {
        var old = f[k];
        if(typeof f[k] === 'function') {
            f[k] = (p1, p2, p3, p4, p5, p6, p7, p8, p9) => {
                console.log(((('Called ' + k) + p1) + p2) + p3);
                try {
                    return map[k].apply(
                        null,
                        [p1, p2, p3, p4, p5, p6, p7, p8, p9]);
                } catch (e) {
                    console.log(e);
                    return old.apply(
                        null,
                        [p1, p2, p3, p4, p5, p6, p7, p8, p9]);
                }
            };
        }
    });
    return f;
};

var mountAndRewireAngular = (memfs) => {
    //var MemoryFileSystem = require("memory-fs");
    //var memfs = new MemoryFileSystem(data);
    var ng = rewire('@angular/cli');
    var map = {
        fs: memfs,
        typescript: {
            sys: {
                readFile: memfs.readFileSync,
                writeFile: memfs.writeFileSync,
                fileExists: memfs.fileExistsSync
            }
        }
    };
    Object.keys(map).forEach(k => {
        var mock = mockAllWithFallback(k, map[k]);
        ng.__set__(k, mock);
    });
    return ng;
};
(mountAndRewireAngular);



[Function: mountAndRewireAngular]